In [63]:
import warnings
import sqlalchemy as db
import pandas as pd
import scipy.interpolate as interpolate
import numpy as np

import plotly.express as px
import plotly.offline
import plotly.graph_objects as go

from nelson_siegel_svensson import NelsonSiegelSvenssonCurve
from nelson_siegel_svensson.calibrate import calibrate_ns_ols

warnings.filterwarnings("ignore")

In [2]:
engine = db.create_engine(
    "mssql+pyodbc:///?odbc_connect=Driver={SQL Server};Server=192.168.11.102;Database=IDPM;Trusted_Connection=True;"
)


In [68]:
yield_curve_id = 575  # 25 is the yield curve id for the EUR curve and 52 is the yield curve id for the USD curve
market_date = 20200101

In [69]:
query = f"""select * from YieldCurvePoint yp 
join YieldCurveData yd
	on yd.YieldCurvePointID = yp.YieldCurvePointID
where yp.YieldCurveID = {yield_curve_id}
and yd.MarketDate = '{market_date}'"""

data = pd.read_sql(
    db.text(query),
    engine.connect(),
)

In [70]:
data.tail()

,YieldCurvePointID,YieldCurveID,RIC,YieldCurveTenorID,YieldCurvePointID,MarketDate,Yield,Discount,DaysToMaturity
6,11070,575,BUSD5Y=,25,11070,2020-01-01 00:00:00.00,5.841477,0.752637,1827
7,11069,575,BUSD6Y=,29,11069,2020-01-01 00:00:00.00,6.111084,0.700316,2192
8,11068,575,BUSD7Y=,33,11068,2020-01-01 00:00:00.00,6.415608,0.646867,2557
9,11082,575,BUSD8Y=,37,11082,2020-01-01 00:00:00.00,6.699590,0.594825,2924
10,11081,575,BUSD9Y=,41,11081,2020-01-01 00:00:00.00,6.944405,0.546182,3288


In [71]:
data['Yield'][0]

4.27497

In [72]:
data['YearToMaturity'] = data['DaysToMaturity'] / 366
data.head()

,YieldCurvePointID,YieldCurveID,RIC,YieldCurveTenorID,YieldCurvePointID,MarketDate,Yield,Discount,DaysToMaturity,YearToMaturity
0,11075,575,BUSD3M=,6,11075,2020-01-01 00:00:00.00,4.274970,0.989646,91,0.248634
1,11074,575,BUSD6M=,7,11074,2020-01-01 00:00:00.00,5.454004,0.973938,182,0.497268
2,11073,575,BUSD1Y=,9,11073,2020-01-01 00:00:00.00,8.284712,0.923491,366,1.000000
3,11083,575,BUSD2Y=,13,11083,2020-01-01 00:00:00.00,7.274677,0.868470,733,2.002732
4,11072,575,BUSD3Y=,17,11072,2020-01-01 00:00:00.00,5.982933,0.839758,1097,2.997268


## Forward rate calculation

In [73]:
# create forward rate
data['ForwardRate'] = data['Yield'][0]
for i in range(1, len(data['Yield'])):
    data['ForwardRate'][i] = (data['Yield'][i] * data['YearToMaturity'][i] -
                              data['Yield'][i-1] * data['YearToMaturity'][i-1]) \
                            / (data['YearToMaturity'][i] - data['YearToMaturity'][i-1])

In [74]:
data

,YieldCurvePointID,YieldCurveID,RIC,YieldCurveTenorID,YieldCurvePointID,MarketDate,Yield,Discount,DaysToMaturity,YearToMaturity,ForwardRate
0,11075,575,BUSD3M=,6,11075,2020-01-01 00:00:00.00,4.274970,0.989646,91,0.248634,4.274970
1,11074,575,BUSD6M=,7,11074,2020-01-01 00:00:00.00,5.454004,0.973938,182,0.497268,6.633038
2,11073,575,BUSD1Y=,9,11073,2020-01-01 00:00:00.00,8.284712,0.923491,366,1.000000,11.084651
3,11083,575,BUSD2Y=,13,11083,2020-01-01 00:00:00.00,7.274677,0.868470,733,2.002732,6.267394
4,11072,575,BUSD3Y=,17,11072,2020-01-01 00:00:00.00,5.982933,0.839758,1097,2.997268,3.381701
5,11071,575,BUSD4Y=,21,11071,2020-01-01 00:00:00.00,5.722129,0.800332,1461,3.991803,4.936135
6,11070,575,BUSD5Y=,25,11070,2020-01-01 00:00:00.00,5.841477,0.752637,1827,4.991803,6.317891
7,11069,575,BUSD6Y=,29,11069,2020-01-01 00:00:00.00,6.111084,0.700316,2192,5.989071,7.460596
8,11068,575,BUSD7Y=,33,11068,2020-01-01 00:00:00.00,6.415608,0.646867,2557,6.986339,8.244421
9,11082,575,BUSD8Y=,37,11082,2020-01-01 00:00:00.00,6.699590,0.594825,2924,7.989071,8.678178


#### with Numpy

In [75]:
arr_maturity = np.array(data['YearToMaturity'].values)
arr_yield = np.array(data['Yield'].values)
arr_forward = np.zeros(len(arr_yield))
arr_forward[0] = arr_yield[0]

In [76]:
arr_maturity

array([0.24863388, 0.49726776, 1.        , 2.00273224, 2.99726776,
       3.99180328, 4.99180328, 5.98907104, 6.9863388 , 7.98907104,
       8.98360656])

In [77]:
for i in range(1, len(arr_yield)):
    arr_forward[i] = (arr_yield[i] * arr_maturity[i] - arr_yield[i-1] * arr_maturity[i-1]) / (arr_maturity[i] - arr_maturity[i-1])

In [78]:
arr_forward

array([ 4.27497   ,  6.633038  , 11.08465143,  6.26739414,  3.38170126,
        4.93613453,  6.31789074,  7.4605963 ,  8.24442062,  8.67817849,
        8.91099582])

In [79]:
# interpolate forward rate
data_array = np.array(data['ForwardRate']/100)
tenor_array = np.array(data['YearToMaturity'])
tenor_array


array([0.24863388, 0.49726776, 1.        , 2.00273224, 2.99726776,
       3.99180328, 4.99180328, 5.98907104, 6.9863388 , 7.98907104,
       8.98360656])

# Cubic Spline

In [80]:
interp_coeff = interpolate.CubicSpline(tenor_array,data_array)

In [81]:
new_tenor = np.linspace(0, np.ceil(data['YearToMaturity'].iloc[-1]), len(data['YearToMaturity']))

In [82]:
cub_inter = interp_coeff(new_tenor)

# NelsonSiegel

In [83]:
data_array,new_tenor

(array([0.0427497 , 0.06633038, 0.11084651, 0.06267394, 0.03381701,
        0.04936135, 0.06317891, 0.07460596, 0.08244421, 0.08678178,
        0.08910996]),
 array([0. , 0.9, 1.8, 2.7, 3.6, 4.5, 5.4, 6.3, 7.2, 8.1, 9. ]))

In [84]:
# NSS model calibrate
curve_fit, status = calibrate_ns_ols(new_tenor, data_array)
curve_fit

NelsonSiegelCurve(beta0=0.6266679440318677, beta1=-0.5616701312323327, beta2=-0.6538565057098971, tau=10.339472502878388)

In [85]:
nelson_new_fit= curve_fit(new_tenor)

In [86]:
fig = px.line()
fig.add_scatter(x=new_tenor, y=nelson_new_fit, mode='lines')
fig.show()

In [21]:
import QuantLib as ql
import pendulum as pl

In [22]:
# remove seconds from the date
market_date = pl.parse(data['MarketDate'][0]).format("YYYY-MM-DD") # type: ignore
market_date

'2020-01-01'

In [23]:
ql.Date(pl.parse(data['MarketDate'][0]).format("YYYY-MM-DD"), '%Y-%m-%d') + 1 # type: ignore


Date(2,1,2020)

In [24]:
data['MarketDate'][0]

'2020-01-01 00:00:00.00'

In [25]:
# create the maturity dates based on DaysToMaturity in data

maturity_dates = [pl.parse(data['MarketDate'][0]) + pl.duration(days=int(d)) for d in data['DaysToMaturity']]
matruity_dates_ql = [ql.Date(data['MarketDate'][0], '%Y-%m-%d') + int(d) for d in data['DaysToMaturity']] # type: ignore
maturity_array = np.array(matruity_dates_ql)
maturity_array, matruity_dates_ql

(array([Date(1,4,2020), Date(1,7,2020), Date(1,1,2021), Date(3,1,2022),
        Date(2,1,2023), Date(1,1,2024), Date(1,1,2025), Date(1,1,2026),
        Date(1,1,2027), Date(3,1,2028), Date(1,1,2029)], dtype=object),
 [Date(1,4,2020),
  Date(1,7,2020),
  Date(1,1,2021),
  Date(3,1,2022),
  Date(2,1,2023),
  Date(1,1,2024),
  Date(1,1,2025),
  Date(1,1,2026),
  Date(1,1,2027),
  Date(3,1,2028),
  Date(1,1,2029)])

In [26]:
maturity_array.shape, cub_inter.shape, nelson_new_fit.shape # type: ignore

((11,), (11,), (11,))

# ForwardCurve

In [27]:
ts = ql.ForwardCurve(maturity_array, nelson_new_fit, ql.ActualActual(ql.ActualActual.Bond))
ts_handle = ql.YieldTermStructureHandle(ts)

In [28]:
calc_date = ql.Date(7,4,2020)
ql.Settings.instance().evaluationDate = calc_date

day_count = ql.ActualActual(ql.ActualActual.Bond)

In [29]:
# # create an array of shape (15,) with the value of 0.06
# test_r = np.full((15,), 7.0)
# ts = ql.ForwardCurve(maturity_array, test_r, ql.ActualActual(ql.ActualActual.Bond))
# ts_handle = ql.YieldTermStructureHandle(ts)

In [30]:
# rate = 0.06
# ts = ql.FlatForward(calc_date, 
#                     rate, 
#                     day_count, 
#                     ql.Compounded, 
#                     ql.Semiannual)
# ts_handle = ql.YieldTermStructureHandle(ts)

In [31]:
callability_schedule = ql.CallabilitySchedule()
call_price = [102.688, 101.792, 100.896, 100.0]
call_date = [ql.Date(14,ql.December,2020), ql.Date(14,ql.December,2021), ql.Date(14,ql.December,2022), ql.Date(14,ql.December,2023)]

for price, date in zip(call_price, call_date):
    callability_schedule.append(ql.Callability(ql.BondPrice(price, ql.BondPrice.Clean), ql.Callability.Call,date))

In [32]:
issue_date = ql.Date(5,ql.December,2014)        
maturity_date = ql.Date(15,ql.December,2024)
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
tenor = ql.Period(ql.Semiannual)
accrual_convention = ql.Unadjusted

schedule = ql.Schedule(issue_date, maturity_date, tenor,
                       calendar, accrual_convention, accrual_convention,
                       ql.DateGeneration.Forward, False)

settlement_days = 2
face_amount = 1000
accrual_daycount = ql.ActualActual(ql.ActualActual.Bond)
coupon = 0.05375

In [33]:

bond = ql.CallableFixedRateBond(
    settlement_days, face_amount,
    schedule, [coupon], accrual_daycount,
    ql.Following, face_amount, issue_date,
    callability_schedule)

In [34]:
def value_bond(a, s, grid_points, bond):
    model = ql.HullWhite(ts_handle, a, s)
    engine = ql.TreeCallableFixedRateBondEngine(model, grid_points)
    bond.setPricingEngine(engine)
    return bond

value_bond(0.07881, 0.00736, 400, bond)
print("Bond price: ",bond.cleanPrice())

Bond price:  96.05823666248934


In [35]:
for c in bond.callability():
    print(f"{c.date()}")

December 14th, 2020
December 14th, 2021
December 14th, 2022
December 14th, 2023


In [36]:
def db_get_schedule(security_id):
    engine = db.create_engine(
        "mssql+pyodbc:///?odbc_connect=Driver={SQL Server};Server=192.168.11.102;Database=IDPM;Trusted_Connection=True;"
    )
    query = f"""SELECT SecurityID, ScheduleValue, StartDate, EndDate FROM tblSecuritySchedule WHERE SecurityID = {security_id}"""

    data = pd.read_sql(
        db.text(query),
        engine.connect(),
    )
    return data

schedule_raw = db_get_schedule(991698)

In [37]:
testdate = '20200407'
ql.Date(testdate, '%Y%m%d')

Date(7,4,2020)